# Curve Generation


In [3]:
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
F = FiniteField(p)

In [4]:
a = 0
b = 7
E = EllipticCurve(F, [a, b])

In [5]:
G_x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
h = 1 # cofactor
G = B = E.lift_x(G_x)

In [6]:
assert(G.order() == 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141)
assert(E.order() / G.order() == h)

# Key Generation


In [7]:
N = FiniteField(G.order())

def keypair (x = None):
    if x is None:
        x = int(N.random_element(1, N.order() - 1))
    P = x * G
    return (x, P)


# ECDSA 

In [8]:
import hashlib
def sign (m, d):
    e = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    k = int(N.random_element(1, N.order() - 1))
    R = k * G # (k, r) is like a one-time key pair
    r = int(R.xy()[0]) % N.order()
    assert(r % N.order() != 0)
    s = (e + d*r)/k % N.order()
    return [r,s]
    

In [9]:
def verify (sig, m, P):
    r, s = sig
    assert(0 < r < N.order())
    assert(0 < s < N.order())
    e = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    u1 = e / s % N.order()
    u2 = r / s % N.order()
    R = u1 * G + u2 * P
    v = int(R.xy()[0]) % N.order()
    return v == r

# Example: ID based signatures



In [14]:
def idKeypair(id, x = None):
    y = int(hashlib.sha256(id).hexdigest(), 16) % N.order()
    (d, P) = keypair(x)
    z = d * y % N.order()
    Q = y * P
    return (z, Q)

In [15]:
def idVerify(sig, m, P, id):
    y = int(hashlib.sha256(id).hexdigest(), 16) % N.order()
    Q = y * P
    return verify(sig, m, Q)

In [16]:
def idSign(m, x, id):
    y = int(hashlib.sha256(id).hexdigest(), 16) % N.order()
    z = x * y % N.order()
    return sign(m, z)

In [19]:
(z, Q) = idKeypair(b"Emil Bayes")
sig = idSign(b"Hello world", z, b"Emil Bayes")
idVerify(sig, b"Hello world", Q, b"Emil Bayes")

True